In [ ]:
import pyspark
import pyspark.sql.functions as func
import pyspark.sql.types as types
sc = pyspark.SparkContext(master='local[*]', appName='DataFrame')
sqlContext = pyspark.sql.SQLContext(sc)

In [ ]:
import re

# Row is one way to creat DataFrame in Python; this one does not require additional schema
# see docs for details: http://spark.apache.org/docs/latest/api/python/pyspark.sql.html
def from_csv(line):
    # This is just for clarity
    HOOD_COLUMN = 1
    TYPE_COLUMN = 2;
    LAND_AREA_COLUMN = 14;
    GROSS_AREA_COLUMN = 15;
    YEAR_COLUMN = 16;
    PRICE_COLUMN = 19;
    # Split column and pack it into dict
    c = line.split(',')
    row = dict()
    row['hood'] = c[HOOD_COLUMN];
    row['type'] = c[TYPE_COLUMN];
    row['landArea'] = int(re.sub(r'[^\d]', '', c[LAND_AREA_COLUMN]));
    row['grossArea'] = int(re.sub(r'[^\d]', '', c[GROSS_AREA_COLUMN]));
    row['year'] = int(re.sub(r'[^\d]', '', c[YEAR_COLUMN]));
    row['price'] = int(re.sub(r'[^\d]', '', c[PRICE_COLUMN]));
    # Returning a Row object as dict is depricated
    return pyspark.Row(**row)

In [ ]:
sales_rdd = sc.textFile('../../data/rollingsales_bronx.csv').map(lambda line: from_csv(line))
sales = sqlContext.createDataFrame(sales_rdd)
sales.printSchema()
sales.show()

RE 1. Analyse the houses data and count houses, averages of areas, yeas and prices, grouping by neighbourhood and type.

In [ ]:
a1 = sales.groupBy('hood', 'type') \
    .agg(func.count('type').alias('count'),
         func.avg('landArea').alias('avgLandArea'),
         func.avg('year').alias('avgYear'),
         func.avg('price').alias('avgPrice')) \
    .orderBy('hood', 'type')
a1.show()